In [2]:
import networkx as nx
import numpy as np
import math
import matplotlib.pyplot as plt
import random
import time

from aux_functions import *
from generation import generation as gen
from local_search import local_search as ls



In [233]:
n, quota, matrix, dataset = load_dataset('problems/instances/A1/symmetric/10.1.in')

A = np.matrix(matrix,dtype=float)
G = nx.from_numpy_array(A)


for i in G.nodes:
    G.nodes[i]['bonus'] = dataset[i].bonus


# matriz com o custo da aresta representando o loop Aii := Pi
for i in range(0, len(G.nodes)):
    G.add_edge(i, i, weight = dataset[i].penalty)


route = [0, 6, 4, 7, 5]


# for i in range(100000):
#     route_edges = [(route[i-1], route[i])
#                     for i in range(len(route))]

#     # Get the number of nodes in the route
#     num_nodes = len(G.nodes)

#     # Initialize the matrix with zeros
#     edges_incidence = np.zeros((num_nodes, num_nodes))

#     node_incidence =  [1 if i in route else 0 for i in range(len(G.nodes))]

#     # Set the entries in the matrix to 1 for the route edges
#     for edge in route_edges:
#         edges_incidence[edge[0], edge[1]] = 1

#     # xii = 1 - yi
#     for i in G.nodes:
#         edges_incidence[i, i] = 1 - node_incidence[i]

#     cost = sum(G[i][j]['weight'] for i, j in zip(*np.where(edges_incidence == 1)))



In [ ]:
# n, quota, matrix, dataset = load_dataset('problems/instances/A1/symmetric/10.1.in')

# A = np.matrix(matrix,dtype=float)
# G = nx.from_numpy_array(A)

# print(G)
# print(G.edges(data=True))

# start_time = time.perf_counter()

# for i in G.nodes:
#     G.nodes[i]['bonus'] = dataset[i].bonus
#     G.nodes[i]['penalty'] =  dataset[i].penalty
#     G.nodes[i]['id'] = dataset[i].id

# finish_time = time.perf_counter()

# duration = finish_time - start_time

# print('duration - ', duration)

# nx.draw(G, with_labels=True)

In [ ]:
from optimization import solve_pctsp

# node_prizes, node_penalties, matrix, quota = load_other_dataset('problems/other_instances/v20.txt')

# TODO: Investigar problema em instancias mais complexas

node_prizes = list(map(lambda node: node.bonus, dataset))
node_penalties = list(map(lambda node: node.penalty, dataset))

# print(node_prizes)
# print(node_penalties)
# print(matrix)
# print(quota)

tour, cost = solve_pctsp(node_prizes, node_penalties, matrix, quota)

if tour is None:
    print('No solution found.')
else:
    print('Tour:', tour)
    print('Cost:', cost)


In [ ]:
# MEMETICO

with open('memetic_history.txt', 'w') as file:
        file.write('iteration;duration(s);cost;route\n')

        for i in range(1):
            start_time = time.perf_counter()
            route = gen.memetic_algorithm(G, quota)
            finish_time = time.perf_counter()
            cost = route_cost(route, G)

            duration = finish_time - start_time

            route_ids = list(map(lambda node: node['id'], route))

            file.write(f'{i+1};{duration:.2f};{cost};{route_ids}\n')

# start_time = time.perf_counter()
# route = gen.memetic_algorithm(G, quota)
# finish_time = time.perf_counter()
# cost = route_cost(route, G)

# duration = finish_time - start_time
# report(duration, route, G)
# plot(route, G)

In [ ]:
# MEMETICO VNS VND

with open('vns_vnd_memetic_history.txt', 'w') as file:
        file.write('iteration;duration(s);cost;route\n')

        for i in range(1):
            start_time = time.perf_counter()
            route = gen.vns_vnd_memetic_algorithm(G, quota)
            finish_time = time.perf_counter()
            cost = route_cost(route, G)

            duration = finish_time - start_time

            route_ids = list(map(lambda node: node['id'], route))

            file.write(f'{i+1};{duration:.2f};{cost};{route_ids}\n')

# start_time = time.perf_counter()
# route = gen.memetic_algorithm(G, quota)
# finish_time = time.perf_counter()
# cost = route_cost(route, G)

# duration = finish_time - start_time
# report(duration, route, G)
# plot(route, G)

In [ ]:
# MEMETICO VNS

with open('vns_memetic_history.txt', 'w') as file:
        file.write('iteration;duration(s);cost;route\n')

        for i in range(1):
            start_time = time.perf_counter()
            route = gen.vns_memetic_algorithm(G, quota)
            finish_time = time.perf_counter()
            cost = route_cost(route, G)

            duration = finish_time - start_time

            route_ids = list(map(lambda node: node['id'], route))

            file.write(f'{i+1};{duration:.2f};{cost};{route_ids}\n')

# start_time = time.perf_counter()
# route = gen.memetic_algorithm(G, quota)
# finish_time = time.perf_counter()
# cost = route_cost(route, G)

# duration = finish_time - start_time
# report(duration, route, G)
# plot(route, G)

In [ ]:
# MEMETICO VND

with open('vnd_memetic_history.txt', 'w') as file:
        file.write('iteration;duration(s);cost;route\n')

        for i in range(1):
            start_time = time.perf_counter()
            route = gen.vnd_memetic_algorithm(G, quota)
            finish_time = time.perf_counter()
            cost = route_cost(route, G)

            duration = finish_time - start_time

            route_ids = list(map(lambda node: node['id'], route))

            file.write(f'{i+1};{duration:.2f};{cost};{route_ids}\n')

# start_time = time.perf_counter()
# route = gen.memetic_algorithm(G, quota)
# finish_time = time.perf_counter()
# cost = route_cost(route, G)

# duration = finish_time - start_time
# report(duration, route, G)
# plot(route, G)

In [ ]:
# GENETICO

with open('genetic_history.txt', 'w') as file:
        file.write('iteration;duration(s);cost;route\n')

        for i in range(1):
            start_time = time.perf_counter()
            route = gen.genetic_algorithm(G, quota)
            finish_time = time.perf_counter()
            cost = route_cost(route, G)

            duration = finish_time - start_time

            route_ids = list(map(lambda node: node['id'], route))

            file.write(f'{i+1};{duration:.2f};{cost};{route_ids}\n')


# start_time = time.perf_counter()
# route = gen.genetic_algorithm(G, quota)
# finish_time = time.perf_counter()
# cost = route_cost(route, G)

# duration = finish_time - start_time
# report(duration, route, G)
# plot(route, G)

In [ ]:
# GRASP

with open('grasp_history.txt', 'w') as file:
        file.write('iteration;duration(s);cost;route\n')

        for i in range(1):
            start_time = time.perf_counter()
            route = gen.grasp_algorithm(G, quota)
            finish_time = time.perf_counter()
            cost = route_cost(route, G)

            duration = finish_time - start_time

            route_ids = list(map(lambda node: node['id'], route))

            file.write(f'{i+1};{duration:.2f};{cost};{route_ids}\n')

# start_time = time.perf_counter()

# route = gen.grasp_algorithm(G, quota)
    
# finish_time = time.perf_counter()
# duration = finish_time - start_time
# report(duration, route, G)
# plot(route, G)



In [ ]:
# ADAPTATIVE GRASP

with open('reactive_grasp_history.txt', 'w') as file:
        file.write('iteration;duration(s);cost;route\n')

        for i in range(1):
            start_time = time.perf_counter()
            route = gen.reactive_grasp_algorithm(G, quota)
            finish_time = time.perf_counter()
            cost = route_cost(route, G)

            duration = finish_time - start_time

            route_ids = list(map(lambda node: node['id'], route))

            file.write(f'{i+1};{duration:.2f};{cost};{route_ids}\n')

# start_time = time.perf_counter()

# route = gen.adaptative_grasp_algorithm(G, quota)

# finish_time = time.perf_counter()
# duration = finish_time - start_time
# report(duration, route, G)
# plot(route, G)

In [ ]:
# GRASP VNS VND

with open('grasp_vns_vnd_history.txt', 'w') as file:
        file.write('iteration;duration(s);cost;route\n')

        for i in range(1):
            start_time = time.perf_counter()
            route = gen.grasp_vns_vnd(G, quota)
            finish_time = time.perf_counter()
            cost = route_cost(route, G)

            duration = finish_time - start_time

            route_ids = list(map(lambda node: node['id'], route))

            file.write(f'{i+1};{duration:.2f};{cost};{route_ids}\n')


In [ ]:
# cost = route_cost(route, G)
# bonus = calculate_bonus_colected(route, G)
# print('cost', cost)
# print('bonus', bonus)